In [5]:
import numpy as np

# ===== 1. 파일 로드 =====
print("=== 파일 로드 ===")
X = np.load('./data/output/tcn_input_combined.npy')
y = np.load('./data/output/tcn_labels.npy')

# ===== 2. 기본 정보 출력 =====
print(f"X shape: {X.shape}  # (윈도우 수, 시간스텝 수, 전체 피처 수)")
print(f"y shape: {y.shape}  # (윈도우 수,)")

# ===== 3. NaN/inf 여부 확인 =====
print("\n=== 결측값 및 무한대 값 검증 ===")
nan_count_X = np.isnan(X).sum()
nan_count_y = np.isnan(y).sum()
inf_count_X = np.isinf(X).sum()
inf_count_y = np.isinf(y).sum()

print(f"X NaN 개수: {nan_count_X:,}")
print(f"X inf 개수: {inf_count_X:,}")
print(f"y NaN 개수: {nan_count_y:,}")
print(f"y inf 개수: {inf_count_y:,}")

# ===== 4. 값 범위 간단 확인 =====
print("\n=== 값 범위 ===")
print(f"X min: {np.nanmin(X):.4f}, X max: {np.nanmax(X):.4f}")
print(f"y unique 값: {np.unique(y)}")

# ===== 5. 샘플 데이터 출력 =====
print("\n=== 샘플 확인 (첫 윈도우 첫 타임스텝) ===")
print(X[0, 0, :])
print(f"y[0]: {y[0]}")



=== 파일 로드 ===
X shape: (20717, 18, 32)  # (윈도우 수, 시간스텝 수, 전체 피처 수)
y shape: (20717,)  # (윈도우 수,)

=== 결측값 및 무한대 값 검증 ===
X NaN 개수: 1,796,813
X inf 개수: 0
y NaN 개수: 0
y inf 개수: 0

=== 값 범위 ===
X min: 0.0000, X max: 250.0000
y unique 값: [0 1]

=== 샘플 확인 (첫 윈도우 첫 타임스텝) ===
[146.5 140.  140.  153.   80.5  79.   79.   82.   94.5  92.   92.   97.
 123.  123.  123.  123.    nan   nan  96.   96.   96.   96.   11.   11.
  11.   11.    nan   nan   nan  61.    1.    0. ]
y[0]: 0


In [6]:
import numpy as np
import pandas as pd

print("===== NaN 결측값 확인 =====")

# 시퀀스 데이터 로드
sequences = np.load('./data/output/tcn_input_combined.npy')
print(f"시퀀스 데이터 형태: {sequences.shape}")

# 1. NaN 값 존재 여부 확인
has_nan = np.isnan(sequences).any()
print(f"NaN 값 존재 여부: {has_nan}")

if has_nan:
    # 2. NaN 값의 개수와 비율
    total_values = sequences.size
    nan_count = np.isnan(sequences).sum()
    nan_ratio = nan_count / total_values * 100
    
    print(f"전체 값 개수: {total_values:,}")
    print(f"NaN 값 개수: {nan_count:,}")
    print(f"NaN 비율: {nan_ratio:.2f}%")
    
    # 3. 각 차원별 NaN 분포
    print(f"\n=== 차원별 NaN 분포 ===")
    print(f"윈도우별 NaN 개수 (상위 5개):")
    nan_per_window = np.isnan(sequences).sum(axis=(1,2))
    top_5_windows = np.argsort(nan_per_window)[-5:][::-1]
    for i, window_idx in enumerate(top_5_windows):
        print(f"  윈도우 {window_idx}: {nan_per_window[window_idx]:,}개")
    
    print(f"\n시간별 NaN 개수 (상위 5개):")
    nan_per_timestep = np.isnan(sequences).sum(axis=(0,2))
    top_5_timesteps = np.argsort(nan_per_timestep)[-5:][::-1]
    for i, time_idx in enumerate(top_5_timesteps):
        print(f"  시간 {time_idx}: {nan_per_timestep[time_idx]:,}개")
    
    print(f"\n피처별 NaN 개수 (상위 5개):")
    nan_per_feature = np.isnan(sequences).sum(axis=(0,1))
    top_5_features = np.argsort(nan_per_feature)[-5:][::-1]
    for i, feature_idx in enumerate(top_5_features):
        print(f"  피처 {feature_idx}: {nan_per_feature[feature_idx]:,}개")
    
    print(f"\n*** 결론: 마스킹이 필수입니다! ***")
    print("이유: NaN 값이 존재하면 PyTorch/TensorFlow에서 gradient 계산 시 문제가 발생합니다.")
    
else:
    print("\n*** 결론: 마스킹이 필수가 아닙니다 ***")
    print("이유: NaN 값이 없으므로 직접 모델에 입력 가능합니다.")

# 4. 샘플 데이터 확인 (처음 3개 윈도우의 첫 번째 시간 스텝)
print(f"\n=== 샘플 데이터 확인 ===")
print("처음 3개 윈도우의 첫 번째 시간 스텝:")
for i in range(min(3, sequences.shape[0])):
    sample_data = sequences[i, 0, :5]  # 첫 5개 피처만 출력
    print(f"윈도우 {i}: {sample_data}")
    if np.isnan(sample_data).any():
        print(f"  -> NaN 포함!")
    else:
        print(f"  -> 정상 데이터")

print("\n" + "="*50)

===== NaN 결측값 확인 =====
시퀀스 데이터 형태: (20717, 18, 32)
NaN 값 존재 여부: True
전체 값 개수: 11,932,992
NaN 값 개수: 1,796,813
NaN 비율: 15.06%

=== 차원별 NaN 분포 ===
윈도우별 NaN 개수 (상위 5개):
  윈도우 7974: 488개
  윈도우 18409: 396개
  윈도우 3261: 390개
  윈도우 16251: 390개
  윈도우 4722: 380개

시간별 NaN 개수 (상위 5개):
  시간 17: 150,051개
  시간 16: 129,401개
  시간 15: 110,063개
  시간 1: 101,051개
  시간 0: 100,205개

피처별 NaN 개수 (상위 5개):
  피처 28: 271,645개
  피처 27: 271,645개
  피처 26: 271,645개
  피처 16: 267,233개
  피처 17: 267,233개

*** 결론: 마스킹이 필수입니다! ***
이유: NaN 값이 존재하면 PyTorch/TensorFlow에서 gradient 계산 시 문제가 발생합니다.

=== 샘플 데이터 확인 ===
처음 3개 윈도우의 첫 번째 시간 스텝:
윈도우 0: [146.5 140.  140.  153.   80.5]
  -> 정상 데이터
윈도우 1: [103. 103. 103. 103.  62.]
  -> 정상 데이터
윈도우 2: [106. 106. 106. 106.  62.]
  -> 정상 데이터

